<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_02_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 2: Neural Networks with PyTorch**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 2 Material

* Part 2.1: Introduction to Neural Networks with PyTorch
* Part 2.2: Encoding Feature Vectors
* Part 2.3: Controlling Overfitting
* **Part 2.4: Saving and Loading a PyTorch Neural Network**

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    Colab = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    Colab = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_01/class_01_6_image01A.png)

If your GMAIL address does not appear your lesson will **not** be graded.

### Create Custom Functions

Run the cell below to create the function needed for this lesson.

In [ ]:
# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# **Saving and Loading a PyTorch Neural Network**

Complex neural networks will take a _long_ time to train.  It is helpful to be able to save a trained neural network so that you can reload it and using it again--a reloaded neural network will **not** require retraining.  

PyTorch provides the following formats for saving neural networks:

* **State Dict** - Stores the model's state dictionary (weights and biases) in a format that can be easily loaded back into the same model architecture.
* **Full Model** - Stores the complete neural network including both the model architecture and weights in a single file.

Usually, you will want to save using the state dict approach as it's more flexible and memory efficient. The state dict only saves the parameters (weights and biases) while keeping the model class definition separate, allowing you to load the weights into different model architectures if needed.

A primary goal of this lesson is to convince you that a PyTorch model that you "regenerate" from a file that was saved to your Google Drive, is exactly the same (i.e. same accuracy) as the original PyTorch model you created and trained.

### Example 1A: Build, Compile and Train Classification Neural Network

The code in `Example 1A` builds, compiles and trains a neural network called `or_model` that can classify the Quality of an orange based on its physical and chemical characteristics.

The code in the cell below reads the Orange Quality dataset from the course HTTP server and creates a DataFrame called `or_df` (i.e. "orange" DataFrame) using this code snippet:

The code in the cell below reads the Orange Quality dataset and creates a DataFrame:

```python
    or_df = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/orange_quality.csv",
    na_values=['NA', '?'])
```

In order to create a feature vector, the 3 non-numeric columns in the dataset: `Color`, `Variety` and `Blemished` must be pre-processed as follows:
1. **Mapping strings to integers:** This is used to take care of the column `Color` which contains several string values. Here is the code snippet that does the mapping:

```python
    # Map str to int
    or_mapping = {'Orange':0,'Deep Orange':1,
           'Light Orange':2,'Orange-Red':3,
           'Yellow-Orange':4}
    or_df['Color'] = or_df['Color'].map(or_mapping)
```

2. **Normalization:** The following code chunk identifies which columns in `or_df` are numeric and then applies Z-normalization to the numeric values.
    
```python
    # Standardise all numeric column with z‑score
    or_numeric_cols = or_df.select_dtypes(include=['int64', 'float64']).columns
    or_df[or_numeric_cols] = or_df[or_numeric_cols].apply(zscore)
    numeric_cols = or_df.select_dtypes(include=['int64', 'float64']).columns
```

3. **Exclude Columns:** To take care the string values in the columns `Variety` and `Blemished`, we will simply exclude them from the list of columns to be used for creating the X feature vector `or_X`:

```python
    # Generate X-values
    or_X = or_df[['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)',
          'Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)',
          'Color']].values
    or_X = np.asarray(or_X).astype('float32')
```

Since we are building a classification neural network, we will need to one-hot encode the column `Quality (1-5)` which contains the `Y-values` using this code snippet:

```python
    # Generate y-values (one-hot encoding)
    or_dummies = pd.get_dummies(or_df['Quality (1-5)'], dtype=int)
    or_Y = or_dummies.values
    or_Y = np.asarray(or_Y).astype('float32')
```

It should be noted that this column is already numeric, so we are **not** using one-hot encoding to replace string values with integer. Rather, one-hot encoding the `Y-values` is necessary to give them the **correct format** for the neural network.


In [ ]:
# Example 1A: Build, Compile and Train Classification Model (Orange Dataset) - PyTorch Version

# ------------------------------------------------------------
# 0️⃣  Imports
# ------------------------------------------------------------
import pandas as pd
import time
import numpy as np
from scipy.stats import zscore
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------
# 1️⃣  Parameters
# ------------------------------------------------------------
EPOCHS        = 100
PATIENCE      = 10
VERBOSE       = 0     # 0 means no output during training

# ------------------------------------------------------------
# 2️⃣  Load data
# ------------------------------------------------------------
or_df = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/orange_quality.csv",
    na_values=['NA', '?'])

# ------------------------------------------------------------
# 5️⃣  Preprocessing
# ------------------------------------------------------------

# Map str to int
or_mapping = {'Orange':0,'Deep Orange':1,
           'Light Orange':2,'Orange-Red':3,
           'Yellow-Orange':4}
or_df['Color'] = or_df['Color'].map(or_mapping)

# Standardise all numeric column with z‑score
or_numeric_cols = or_df.select_dtypes(include=['int64', 'float64']).columns
or_df[or_numeric_cols] = or_df[or_numeric_cols].apply(zscore)


# Generate X-values
or_X = or_df[['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)',
       'Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)',
        'Color']].values
or_X = np.asarray(or_X).astype('float32')

# Generate Y-values
or_dummies = pd.get_dummies(or_df['Quality (1-5)'], dtype=int) # Classification
or_Y = or_dummies.values
or_Y = np.asarray(or_Y).astype('float32')

# ------------------------------------------------------------
# 6️⃣  Prepare data for PyTorch
# ------------------------------------------------------------

# Convert to PyTorch tensors
or_X_tensor = torch.FloatTensor(or_X)
or_y_tensor = torch.LongTensor(np.argmax(or_Y, axis=1))  # Convert one-hot to class indices

# Split into train and validation sets
or_X_train, or_X_val, or_y_train, or_y_val = train_test_split(
    or_X_tensor, or_y_tensor, test_size=0.2, random_state=42
)

# Create data loaders
or_train_dataset = TensorDataset(or_X_train, or_y_train)
or_val_dataset = TensorDataset(or_X_val, or_y_val)
or_train_loader = DataLoader(or_train_dataset, batch_size=32, shuffle=True)
or_val_loader = DataLoader(or_val_dataset, batch_size=32, shuffle=False)

# ------------------------------------------------------------
# 7️⃣  Define model
# ------------------------------------------------------------

class OrangeQualityModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(OrangeQualityModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)  # No softmax here - it's included in CrossEntropyLoss
        return x

# Create model instance
or_input_dim = or_X_train.shape[1]
or_num_classes = or_Y.shape[1]
or_model = OrangeQualityModel(or_input_dim, or_num_classes)

# ------------------------------------------------------------
# 8️⃣  Define optimizer and loss function
# ------------------------------------------------------------

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(or_model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 9️⃣  Training loop with early stopping and history tracking
# ------------------------------------------------------------

# Initialize history tracking lists
or_train_losses = []
or_val_losses = []
or_train_accuracies = []
or_val_accuracies = []

# Early stopping variables
best_val_acc = 0.0
patience_counter = 0

print(f"------Training Starting for {EPOCHS} epochs --------------")
start_time = time.time()

for epoch in range(EPOCHS):
    # Training phase
    or_model.train()
    or_train_loss = 0.0
    or_train_correct = 0
    or_train_total = 0

    for inputs, labels in or_train_loader:
        optimizer.zero_grad()
        outputs = or_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        or_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        or_train_total += labels.size(0)
        or_train_correct += (predicted == labels).sum().item()

    # Validation phase
    or_model.eval()
    or_val_loss = 0.0
    or_val_correct = 0
    or_val_total = 0

    with torch.no_grad():
        for inputs, labels in or_val_loader:
            outputs = or_model(inputs)
            loss = criterion(outputs, labels)

            or_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            or_val_total += labels.size(0)
            or_val_correct += (predicted == labels).sum().item()

    # Calculate accuracies
    or_train_acc = 100. * or_train_correct / or_train_total
    or_val_acc = 100. * or_val_correct / or_val_total

    # Store history for plotting
    or_train_losses.append(or_train_loss/len(or_train_loader))
    or_val_losses.append(or_val_loss/len(or_val_loader))
    or_train_accuracies.append(or_train_acc)
    or_val_accuracies.append(or_val_acc)

    if VERBOSE > 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}]')
        print(f'Train Loss: {or_train_loss/len(or_train_loader):.4f}, '
              f'Train Acc: {or_train_acc:.2f}%')
        print(f'Val Loss: {or_val_loss/len(or_val_loader):.4f}, '
              f'Val Acc: {or_val_acc:.2f}%')
        print('-' * 50)


    # Early stopping logic
    if or_val_acc > best_val_acc:
        best_val_acc = or_val_acc
        patience_counter = 0
        # Save best model
        torch.save(or_model.state_dict(), "or_best_classification_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

# ---------------------------------------------------------------------------
# 🔟 Inspect training - Now with proper history tracking
# ---------------------------------------------------------------------------

print(f"\nTraining finished.")
print(f"Best validation accuracy: {best_val_acc:.4f}")

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

# Show the best validation accuracy and loss (same as original)
best_val_acc_history = max(or_val_accuracies) if or_val_accuracies else 0.0
best_val_loss_history = min(or_val_losses) if or_val_losses else float('inf')

print(f"Best validation accuracy: {best_val_acc_history:.4f}")
print(f"Best validation loss: {best_val_loss_history:.4f}")

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image01C.png)

The `or_model` neural network trained very quickly (< 1 min) but the best validation accuracy (`val accuracy`) is only about 40-45%. It should also be noted that in this particular run, `Early Stopping` terminated training at the 17th epoch.

### Example 1B: Plot Training History

The code in the cell below generates two side-by-side plots, an **Accuracy Curve** and a **Loss Curve**. These curves provide a visual way to follow what happened during training of `or_model`.

In [ ]:
# Example 1B: Plot Training History

import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))

# Main title above both subplots
plt.suptitle('Training Visualization: Orange Quality Dataset',
             fontsize=16, fontweight='medium')

plt.subplot(1,2,1)
plt.plot(or_train_accuracies, label='train')
plt.plot(or_val_accuracies, label='val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.subplot(1,2,2)
plt.plot(or_train_losses, label='train')
plt.plot(or_val_losses, label='val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image02C.png)


### **Exercise 1A: Build and Train a Classification Neural Network**

In the cell below build and train a new classification neural network called ap_model.

Start by reading the dataset and creating a DataFrame called ap_df ("apple" DataFrame) using this code chunk:

```python

    ap_df = pd.read_csv(
        "https://biologicslab.co/BIO1173/data/apple_quality.csv",
        na_values=['NA', '?'])

```

The goal of your neural network model ap_model will be to classify the apples in the Apple Quality dataset using the values in the following columns: 'Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Acidity' and 'Ripeness'. Since all of these columns are numeric, there is no need pre-process any of these columns. Moreover, the numerical values all have a similar magnitude so you don't need to standardize any column to their z-scores. Nor do you need to split the data into Training/Validation sets or suffle the data. When you generate your X-values, you should called them ap_X.

Since you are building a classification neural network, you will need to One-Hot Encode the column containing the Y-values, Quality. This column is non-numeric, so by One-Hot Encoding it, you will accomplish two things: (1) replace string values with integers and (2) give the Y-values the correct format for the neural network. When you generate your Y-values, you should called them ap_Y.

Again, because you want your ap_model to act as a classifier, use the softmax activation function in the output layer. You should also compile your model using categorical_crossentropy as the loss function.

Finally, train (fit) your model on your X-values (ap_X) and your Y-values (ap_Y) for 100 epochs.

### **Exercise 1: Build and Train a Classification Neural Network**

In the cell below build and train a new classification neural network called `ap_model`.

Start by `copy-and-paste` Example 1 into the cell below.

Read the **`Apple Quality dataset`** and creat a DataFrame called `ap_df` ("apple" DataFrame) using this code chunk:

```python
    ap_df = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/apple_quality.csv",
    na_values=['NA', '?'])
```

The goal of your training your neural network model `ap_model` will be to learn how to predict the classification apples using the values in the following columns: 'Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Acidity' and 'Ripeness'. Since all of these columns are numeric you should standardize their values by converting them to their z-scores. You can simply re-use the code in `Example 1` making sure to change the prefix `or_` to `ap_`.

Again, since all of these columns are numeric, there is no need map any strings to integers as was done in `Example 1`. The easiest and _safest_ way to accomplish this is to simply comment-out the unecessary code in `Example 1` as follows:


```python
    # Map str to int
    #mapping = {'Orange':0,'Deep Orange':1,
    #           'Light Orange':2,'Orange-Red':3,
    #           'Yellow-Orange':4}
    #or_df['Color'] = or_df['Color'].map(mapping)
```
Commenting-out code instead of simply deleting it has the distinct advantage that if you make a mistake, is relatively easy to correct it by simply adding (or removing) the **`#`** before the line.


To generate your X feature vector change this code chunk

```python
    # Generate X-values
    or_X = or_df[['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)',
    'Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)',
    'Color']].values
    or_X = np.asarray(or_X).astype('float32')
```
to read as

```python
    # Generate X‑values
    ap_X = ap_df[['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Acidity', 'Ripeness']].values
    ap_X = np.asarray(ap_X).astype('float32')
```

This will make sure on the correct columns are used to generate your `X` feature vector.

Since you are building a classification neural network, you will need to one-Hot encode the column called `Quality` containing your `Y-values`. You can simple re-use the code in `Example 1`, making sure to use the correct name for the target column (i.e `Quality`).

Finally, train (fit) your model on your X-values (`ap_X`) and your Y-values (`ap_Y`) for 100 epochs with verbose set to `0`.  

In [ ]:
# Insert your code for Exercise 1A here




If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image03C.png)

Your `ap_model` neural network appears to have done a much better job with a best validation accuracy (`val accuracy`) over 90%. It should also be noted that in this particular run, `Early Stopping` terminated training at the 32rd epoch.

### **Exercise 1B: Visualize Training**

In the cell below write the code to generate two side-by-side plots, an **Accuracy Curve** and a **Loss Curve** for your `ap_model`.

In [ ]:
# Insert your code for Exercise 1B here



If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image04C.png)


----------------------------------------
## **Time and Cost of Training Large Language Models (LLMs)**

Large Language Models (LLMs) require a lot of time and money to train. Here is some of the available data as of September 2025.

#### **Largest LLMs (as of September 2025)**

| Model | Approx. size (parameters) | Rough training duration | Rough training cost | Source |
|-------|---------------------------|-------------------------|---------------------|--------|
| **PaLM 2‑G** (Google) | **540 B** | ~4 months on ≈ 5 000–10 000 GPUs (≈ 2 million GPU‑hours) | **$200 M – $250 M** | Google AI blog (2024), “PaLM 2: Language Models for the Web” |
| **GPT‑4** (OpenAI) | **175 B** (largest released variant) | ~3 months on ≈ 10 000–15 000 GPUs (≈ 1.3 million GPU‑hours) | **$30 M – $50 M** | OpenAI press release (2023), estimates from *Bloomberg* and *The Verge* |
| **Claude 3** (Anthropic) | **200 B** | ~3 months on ≈ 8 000 GPUs (≈ 1 million GPU‑hours) | **$30 M – $60 M** | Anthropic blog (2024) |
| **LLaMA‑2‑70B** (Meta) | **70 B** | ~1 month on ≈ 1 500 GPUs (≈ 0.2 million GPU‑hours) | **$5 M – $10 M** | Meta AI research paper (2023) |


####**Quick take-aways**

1. **Largest publicly‑known LLM (as of 2025):**  
   *PaLM 2‑G* – **540 billion** parameters, the only model known to exceed the 175‑B‑parameter range of GPT‑4.

2. **Training time:**  
   Even the smallest “state-of-the-art” models require **weeks to months** on **thousands of GPUs**.  
   * PaLM 2-G ≈ **2 million GPU-hours** → ~4 months on a 5 000‑GPU cluster.

3. **Training cost:**  
   Costs run in the **tens to hundreds of millions** of dollars.  
   * 175B-parameter models: **\$30-50M**  
   * 540B-parameter models: **\$200-250M**

> **Bottom line:**  
> The field is rapidly moving toward ever larger models, but the practical ceiling is still in the *hundreds of billions* of parameters.  Training such a model is a **multi-month, multi‑million‑GPU‑hour operation** that costs **\$30-250 million**, depending on size and hardware budget.

---------------------


### Example 2: Determine the Model's RMSE and Accuracy

The overall objective of this assignment is to convince you that you can save a trained neural network to a file, and then later, recreate the neural network from the file, without changing the model's accuracy.

**Why is this important?**

As you already know, it can take significant time and processing power to train even relatively small neural networks that we created so far in this course. Neural networks that are used commercially (think "Siri" or "Alexa" or ChatGPT) are many times larger and require enormous resources as well as weeks (or months) to train. Obviously, if you had to train a neural network every time you wanted to use it, it won't be very practical and there would be little interest in "AI". However, once the neural network has been trained, you can save it to a file, and then re-use it over and over again, without any loss in the neural network's ability to solve problems (i.e. loss in accuracy).

The code in the cell below calculates the ability of the `or_model` neural network to predict an orange's quality (Y-value) based on its physical and chemical characteristics (X-values). Two measures of predictive ability are computed, the Root Mean Square Error (RMSE) and Accuracy. The code stores the RMSE value in the variable `or_Score` and the Accuracy value in the variable `or_Correct` and then prints out these values.


### Example 2: Determine the Model's RMSE and Accuracy

The code in the cell below calculates 4 **accuracy metrics** about the `or_model` before we save it to disk.

In [ ]:
# Example 2: Determine the model's accuracy before saving to disk

import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Print title
print("Measurements of Orange NN model `or_model`")
print("--------------------------------------------")

# Measure RMSE error (using cross-entropy loss as proxy since we're doing classification)
# For classification, we typically don't use RMSE directly, but here's how you'd calculate it:
with torch.no_grad():
    or_Pred = or_model(or_X_tensor).numpy()  # Get predictions from model
    or_Score = np.sqrt(metrics.mean_squared_error(or_Pred, or_Y))
    print(f"Before save score (RMSE): {or_Score}")

# Measure the accuracy
or_Predict_classes = np.argmax(or_Pred,axis=1)
or_Expected_classes = np.argmax(or_Y,axis=1)
or_Correct = accuracy_score(or_Expected_classes,or_Predict_classes)
print(f"Before save Accuracy: {or_Correct}")

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image05C.png)

### **Exercise 2: Determine the Model's Accuracy**

In the cell below, write the code to calculate the 4 **accuracy metrics** shown in `Example 2` about your `ap_model` before we save it to disk.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see something _similar_ to the following output:

![__](https://biologicslab.co/BIO1173/images/class_02/class_02_4_image06C.png)

According to the output shown above, your `ap_model` is better than 95% accurate when it comes to predicting apple quality. Apparently, it's a little easier to predict an apple's `Quality` with a classification neural network than to predict orange quality.

### Example 3: Save the Model

The code in the cell below saves the trained neural network `or_model` as a file in two different file formats: PyTorch state dict and full model.

Each file is saved in the current working directory (save_path = "."). The filename of the state dict file is `or_model_pytorch.pth` while the filename of the full model file is `or_full_model_pytorch.pth`.

In [ ]:
# Example 3: Save the model

import os
import torch

# Save path is the current directory
save_path = "."

# Save the PyTorch model state dict (weights and architecture)
torch.save(or_model.state_dict(), os.path.join(save_path, "or_model_pytorch.pth"))

# Save the entire model (architecture + weights)
torch.save(or_model, os.path.join(save_path, "or_full_model_pytorch.pth"))

# Print out the files in current directory
files = os.listdir()
print(files)


If your code is correct you should see something _similar_ to the following output:

```text
['.config', 'or_full_model_pytorch.pth', 'ap_best_classification_model.pth', 'or_model_pytorch.pth', 'drive', 'or_best_classification_model.pth', 'sample_data']
```
After running the code cell above, there should now be two new files in your current directory, `or_model_pytorch.pth` and `or_full_model_pytorch.pth`.

### **Exercise 3: Save the Model**

In the code cell below save your _trained_ neural network `ap_model` as a JSON file with the filename, `ap_model.json`, and as a native Keras file with the filenmane `ap_model.keras`. Save both files to your current working directory (`save_path = "."`).

In [ ]:
# Insert your code for Exercise 3 here



You should now see the two more files with your neural network, `ap_model_pytorch.pth` and `ap_full_model_pytorch.pth`.

### Example 4: Create New Model from Saved Model

Once a trained model has been saved, it is a simple matter to read the file to make an exact copy of the model. In PyTorch, we need to recreate the model architecture first and then load the saved weights into it.

In Example 4, we have recreated the neural network architecture and loaded the saved weights into a new model instance called `or2_model` to differentiate it from the one that was built previously.

In [ ]:
# Example 4: Create new model from saved model

import torch
from torch import nn
import os

print("=== Model Loading Verification ===")

# Print details of model
print(f"Training data shapes:")
print(f"X shape: {or_X.shape}")  # Should be (samples, 8)
print(f"Y shape: {or_Y.shape}")  # This is 8

# Check dimensions
num_classes = or_Y.shape[1]  # This is 8 - so we need to create a model with 8 output classes
input_dim = or_X.shape[1]   # This is 8 - 8 input features

print(f"Input features: {input_dim}")
print(f"Number of classes: {num_classes}")

# Recreate the EXACT same model class structure as used in training
class OrangeQualityModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(OrangeQualityModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Create model with EXACT same dimensions as training
or2_model = OrangeQualityModel(input_dim, num_classes)  # input_dim=8, num_classes=8

try:
    print("Loading weights from saved model...")
    or2_model.load_state_dict(torch.load("or_model_pytorch.pth"))
    print("✓ Orange Quality model loaded successfully!")

except Exception as e:
    print(f"✗ Error loading model: {e}")

If your code is correct you should see the following output:

![__](https://biologicslab.co/BIO1173/images/class_02/class_02_4_image07C.png)

### **Exercise 4: Create New Model from Saved Model**

In the cell below create a new neural network called `ap2_model` from the file `ap_model_pytorch.pth` in your current directory. Print out a summary of your new `ap2_model`.

**Code Hints:**

Change the class name `OrangeQualityModel` to read `AppleQualityModel`.

In [ ]:
# Insert your code for Exercise 4 here



If your code is correct you should see the following output:

![__](https://biologicslab.co/BIO1173/images/class_02/class_02_4_image08C.png)

### Example 5: Compare the Predictive Accuracy of the Old and New Models

The code in the cell below computes the RMSE error and the Accuracy of our new model `or2_model` and compares these values with the original `or_model`. We are trying to address the question whether re-loaded model has the same accuracy as the original model?

In [ ]:
# Example 5: Compare the Predictive Accuracy of the Old and New Models

# Set both models to evaluation mode for consistent predictions
or_model.eval()  # Make sure original is in eval mode
or2_model.eval()  # Make sure new model is also in eval mode

print("\n=== MODEL VERIFICATION ===")
with torch.no_grad():
    # Get predictions from both models (both should be in eval mode now)
    or_pred = or_model(or_X_tensor).numpy()
    or2_pred = or2_model(or_X_tensor).numpy()

    print(f"Original model prediction shape: {or_pred.shape}")
    print(f"New model prediction shape: {or2_pred.shape}")

    # Show first few predictions for comparison
    orig_preds = np.argmax(or_pred, axis=1)
    new_preds = np.argmax(or2_pred, axis=1)

    print(f"\nOriginal model (first 5): {orig_preds[:5]}")
    print(f"New model      (first 5): {new_preds[:5]}")

    # Check exact match
    are_equal = np.array_equal(orig_preds, new_preds)
    print(f"\nPredictions identical: {are_equal}")

    if not are_equal:
        diff_count = np.sum(orig_preds != new_preds)
        print(f"Number of different predictions: {diff_count}")

        # Show some examples where they differ
        diff_indices = np.where(orig_preds != new_preds)[0]
        if len(diff_indices) > 0:
            print("First few differing indices:", diff_indices[:5])
            for i in range(min(5, len(diff_indices))):
                idx = diff_indices[i]
                print(f"Index {idx}: Original={orig_preds[idx]}, New={new_preds[idx]}")

print("\n=== Model Architecture Check ===")
print("Original model:")
print(or_model)
print("\nNew loaded model:")
print(or2_model)

# Let's also check the weights are identical
print("\n=== Weight Comparison ===")
weight_diffs = []
for (name1, param1), (name2, param2) in zip(or_model.named_parameters(), or2_model.named_parameters()):
    diff = torch.sum(torch.abs(param1 - param2)).item()
    weight_diffs.append(diff)
    print(f"{name1}: difference = {diff:.2e}")

total_weight_diff = sum(weight_diffs)
print(f"\nTotal weight difference: {total_weight_diff:.2e}")


If the code is correct the output you should see something _similar_ to the following output:`

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image09C.png)

As you can see, there is **_no difference_** in the accuracy of the saved model compared to the original one.

>> ### **_Train Once_...Use Anywhere!**

Big generative models like `ChatGTP` can take days or even months to train, but once they are trained and saved, they can process new data very fast at very little cost.

### **Exercise 5: Compare the Predictive Accuracy of the Old and New Models**

In the cell below write the code to compute the RMSE and Accuracy values for your `ap2Model` and print out these values along with the values for your original `ap_model`.

In [ ]:
# Insert your code for Exercise 5 here




If the code is correct, you should see something _similar_ to the following output:

![__](https://biologicslab.co:/BIO1173/images/class_02/class_02_4_image10C.png)

## **Summary: Pre-trained Model Persistence and Its Importance for Large Language Models**

The ability to save and reload trained PyTorch neural networks without any loss of accuracy represents a fundamental breakthrough in machine learning workflow efficiency. When we train a complex neural network, especially one with millions or billions of parameters like modern large language models (LLMs), the training process can take weeks or months using powerful computational resources. However, once training is complete, we can save the model's weights and architecture to disk, creating a portable file that captures all the learned knowledge.

This capability becomes critically important in the era of extremely large LLMs because these models require enormous computational resources for training - often costing thousands of dollars in cloud computing time and consuming massive amounts of energy. By saving trained models, practitioners can:
1. Avoid retraining identical or similar architectures from scratch
2. Deploy models to production environments immediately
3. Share pre-trained models with others in the research community
4. Fine-tune existing models on new datasets without starting from random weights

For LLMs specifically, this persistence allows researchers and developers to build upon existing knowledge rather than reinventing the wheel each time. A model trained on general text can be saved, then fine-tuned for specific applications like medical diagnosis, legal document analysis, or creative writing - all while maintaining the original model's accuracy and performance characteristics. This approach democratizes access to sophisticated AI capabilities by making powerful pre-trained models accessible to organizations that cannot afford expensive training infrastructure, ultimately accelerating innovation in artificial intelligence.


# **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook if you are using a Mac. If you are using Windows, use **File --> Print.. --> Microsoft Print to PDF** to generate your PDF. Name your PDF `Class_02_4.lastname.pdf` where _lastname_ is your last name, and upload your file to Canvas.

## **Lizard Tail**


## **Stable Diffusion**

![___](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Astronaut_Riding_a_Horse_%28SD3.5%29.webp/1024px-Astronaut_Riding_a_Horse_%28SD3.5%29.webp.png)

>*An image generated with Stable Diffusion 3.5 based on the text prompt*

**Stable Diffusion** is a deep learning, text-to-image model released in 2022 based on diffusion techniques. The generative artificial intelligence technology is the premier product of `Stability AI` and is considered to be a part of the ongoing artificial intelligence boom.

It is primarily used to generate detailed images conditioned on text descriptions, though it can also be applied to other tasks such as inpainting, outpainting, and generating image-to-image translations guided by a text prompt.[3] Its development involved researchers from the CompVis Group at Ludwig Maximilian University of Munich and Runway with a computational donation from Stability and training data from non-profit organizations.

**Stable Diffusion** is a latent diffusion model, a kind of deep generative artificial neural network. Its code and model weights have been released publicly, and it can run on most consumer hardware equipped with a modest GPU with at least 4 GB VRAM. This marked a departure from previous proprietary text-to-image models such as DALL-E and Midjourney which were accessible only via cloud services.

**Development**

Stable Diffusion originated from a project called Latent Diffusion, developed in Germany by researchers at Ludwig Maximilian University in Munich and Heidelberg University. Four of the original 5 authors (Robin Rombach, Andreas Blattmann, Patrick Esser and Dominik Lorenz) later joined Stability AI and released subsequent versions of Stable Diffusion.

The technical license for the model was released by the CompVis group at Ludwig Maximilian University of Munich. Development was led by Patrick Esser of Runway and Robin Rombach of CompVis, who were among the researchers who had earlier invented the latent diffusion model architecture used by Stable Diffusion. Stability AI also credited EleutherAI and LAION (a German nonprofit which assembled the dataset on which Stable Diffusion was trained) as supporters of the project.

**Technology**

The denoising process used by Stable Diffusion. The model generates images by iteratively denoising random noise until a configured number of steps have been reached, guided by the CLIP text encoder pretrained on concepts along with the attention mechanism, resulting in the desired image depicting a representation of the trained concept.

**Architecture**

Diffusion models, introduced in 2015, are trained with the objective of removing successive applications of Gaussian noise on training images, which can be thought of as a sequence of denoising autoencoders. The name diffusion is from the thermodynamic diffusion, since they were first developed with inspiration from thermodynamics.

Models in Stable Diffusion series before SD 3 all used a variant of diffusion models, called latent diffusion model (LDM), developed in 2021 by the CompVis (Computer Vision & Learning) group at LMU Munich.

Stable Diffusion consists of 3 parts: the variational autoencoder (VAE), U-Net, and an optional text encoder. The VAE encoder compresses the image from pixel space to a smaller dimensional latent space, capturing a more fundamental semantic meaning of the image. Gaussian noise is iteratively applied to the compressed latent representation during forward diffusion. The U-Net block, composed of a ResNet backbone, denoises the output from forward diffusion backwards to obtain a latent representation. Finally, the VAE decoder generates the final image by converting the representation back into pixel space.

The denoising step can be flexibly conditioned on a string of text, an image, or another modality. The encoded conditioning data is exposed to denoising U-Nets via a cross-attention mechanism. For conditioning on text, the fixed, pretrained CLIP ViT-L/14 text encoder is used to transform text prompts to an embedding space. Researchers point to increased computational efficiency for training and generation as an advantage of LDMs.

With 860 million parameters in the U-Net and 123 million in the text encoder, Stable Diffusion is considered relatively lightweight by 2022 standards, and unlike other diffusion models, it can run on consumer GPUs, and even CPU-only if using the OpenVINO version of Stable Diffusion.

**SD XL**
The XL version uses the same LDM architecture as previous versions, except larger: larger UNet backbone, larger cross-attention context, two text encoders instead of one, and trained on multiple aspect ratios (not just the square aspect ratio like previous versions).

The SD XL Refiner, released at the same time, has the same architecture as SD XL, but it was trained for adding fine details to preexisting images via text-conditional img2img.

**SD 3.0**

Main article: Diffusion model § Rectified flow
The 3.0 version completely changes the backbone. Not a UNet, but a Rectified Flow Transformer, which implements the rectified flow method with a Transformer.

The Transformer architecture used for SD 3.0 has three "tracks", for original text encoding, transformed text encoding, and image encoding (in latent space). The transformed text encoding and image encoding are mixed during each transformer block.

The architecture is named "multimodal diffusion transformer (MMDiT), where the "multimodal" means that it mixes text and image encodings inside its operations. This differs from previous versions of DiT, where the text encoding affects the image encoding, but not vice versa.

**Training data**

Stable Diffusion was trained on pairs of images and captions taken from LAION-5B, a publicly available dataset derived from Common Crawl data scraped from the web, where 5 billion image-text pairs were classified based on language and filtered into separate datasets by resolution, a predicted likelihood of containing a watermark, and predicted "aesthetic" score (e.g. subjective visual quality). The dataset was created by LAION, a German non-profit which receives funding from Stability AI. The Stable Diffusion model was trained on three subsets of LAION-5B: laion2B-en, laion-high-resolution, and laion-aesthetics v2 5+. A third-party analysis of the model's training data identified that out of a smaller subset of 12 million images taken from the original wider dataset used, approximately 47% of the sample size of images came from 100 different domains, with Pinterest taking up 8.5% of the subset, followed by websites such as WordPress, Blogspot, Flickr, DeviantArt and Wikimedia Commons. An investigation by Bayerischer Rundfunk showed that LAION's datasets, hosted on Hugging Face, contain large amounts of private and sensitive data.

**Training procedures**

The model was initially trained on the laion2B-en and laion-high-resolution subsets, with the last few rounds of training done on LAION-Aesthetics v2 5+, a subset of 600 million captioned images which the LAION-Aesthetics Predictor V2 predicted that humans would, on average, give a score of at least 5 out of 10 when asked to rate how much they liked them. The LAION-Aesthetics v2 5+ subset also excluded low-resolution images and images which LAION-5B-WatermarkDetection identified as carrying a watermark with greater than 80% probability. Final rounds of training additionally dropped 10% of text conditioning to improve Classifier-Free Diffusion Guidance.

The model was trained using 256 Nvidia A100 GPUs on Amazon Web Services for a total of 150,000 GPU-hours, at a cost of $600,000.

![__](https://upload.wikimedia.org/wikipedia/commons/f/f6/Stable_Diffusion_architecture.png)

>Diagram of the latent diffusion architecture used by Stable Diffusion


**Limitations**

Stable Diffusion has issues with degradation and inaccuracies in certain scenarios. Initial releases of the model were trained on a dataset that consists of 512×512 resolution images, meaning that the quality of generated images noticeably degrades when user specifications deviate from its "expected" 512×512 resolution; the version 2.0 update of the Stable Diffusion model later introduced the ability to natively generate images at 768×768 resolution. Another challenge is in generating human limbs due to poor data quality of limbs in the LAION database. The model is insufficiently trained to replicate human limbs and faces due to the lack of representative features in the database, and prompting the model to generate images of such type can confound the model. Stable Diffusion XL (SDXL) version 1.0, released in July 2023, introduced native 1024x1024 resolution and improved generation for limbs and text.

Accessibility for individual developers can also be a problem. In order to customize the model for new use cases that are not included in the dataset, such as generating anime characters ("waifu diffusion"), new data and further training are required. Fine-tuned adaptations of Stable Diffusion created through additional retraining have been used for a variety of different use-cases, from medical imaging to algorithmically generated music. However, this fine-tuning process is sensitive to the quality of new data; low resolution images or different resolutions from the original data can not only fail to learn the new task but degrade the overall performance of the model. Even when the model is additionally trained on high quality images, it is difficult for individuals to run models in consumer electronics. For example, the training process for waifu-diffusion requires a minimum 30 GB of VRAM, which exceeds the usual resource provided in such consumer GPUs as Nvidia's GeForce 30 series, which has only about 12 GB.

The creators of Stable Diffusion acknowledge the potential for algorithmic bias, as the model was primarily trained on images with English descriptions. As a result, generated images reinforce social biases and are from a western perspective, as the creators note that the model lacks data from other communities and cultures. The model gives more accurate results for prompts that are written in English in comparison to those written in other languages, with western or white cultures often being the default representation.

**End-user fine-tuning**

To address the limitations of the model's initial training, end-users may opt to implement additional training to fine-tune generation outputs to match more specific use-cases, a process also referred to as personalization. There are three methods in which user-accessible fine-tuning can be applied to a Stable Diffusion model checkpoint:

An "embedding" can be trained from a collection of user-provided images, and allows the model to generate visually similar images whenever the name of the embedding is used within a generation prompt.[44] Embeddings are based on the "textual inversion" concept developed by researchers from Tel Aviv University in 2022 with support from Nvidia, where vector representations for specific tokens used by the model's text encoder are linked to new pseudo-words. Embeddings can be used to reduce biases within the original model, or mimic visual styles.

A "hypernetwork" is a small pretrained neural network that is applied to various points within a larger neural network, and refers to the technique created by NovelAI developer Kurumuz in 2021, originally intended for text-generation transformer models. Hypernetworks steer results towards a particular direction, allowing Stable Diffusion-based models to imitate the art style of specific artists, even if the artist is not recognised by the original model; they process the image by finding key areas of importance such as hair and eyes, and then patch these areas in secondary latent space.
DreamBooth is a deep learning generation model developed by researchers from Google Research and Boston University in 2022 which can fine-tune the model to generate precise, personalised outputs that depict a specific subject, following training via a set of images which depict the subject.